# Sentiment Analysis using BERT
Fine-tuning BERT for classifying text into positive, neutral, or negative sentiment.

In [14]:
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.optim import AdamW

from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report


In [15]:
!pip install transformers datasets pandas scikit-learn torch matplotlib tqdm

In [21]:
dataset = load_dataset("imdb")


train_subset = dataset["train"].shuffle(seed=42).select(range(2000))
test_subset = dataset["test"].shuffle(seed=42).select(range(500))
# Convert Column objects to list
train_texts = list(train_subset["text"])
train_labels = list(train_subset["label"])
test_texts = list(test_subset["text"])
test_labels = list(test_subset["label"])



In [22]:


tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


In [23]:


# 1. Load model
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

# 2. Move to device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 3. Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# 4. DataLoaders (keep as you had them)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.train()
for epoch in range(2):  # train for 2 epochs
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} - Training loss: {total_loss/len(train_loader):.4f}")


Epoch 1 - Training loss: 0.4284
Epoch 2 - Training loss: 0.2174


In [25]:
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [26]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install "transformers[torch]"


Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install --upgrade accelerate


Note: you may need to restart the kernel to use updated packages.


In [29]:
print("Test Accuracy:", accuracy_score(true_labels, preds))
print("\nClassification Report:\n", classification_report(
    true_labels, preds, labels=[0, 1], target_names=["negative", "positive"]
))

# -------------------------
# 8. Test on custom sentences
# -------------------------
def predict_sentiment(text):
    encoding = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
    outputs = model(**encoding)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return "positive" if pred == 1 else "negative"

print("\nCustom Predictions:")
print("I loved this movie! ->", predict_sentiment("I loved this movie!"))
print("It was the worst film ever. ->", predict_sentiment("It was the worst film ever."))
print("The acting was okay, but story was boring. ->", predict_sentiment("The acting was okay, but story was boring."))

Test Accuracy: 0.878

Classification Report:
               precision    recall  f1-score   support

    negative       0.89      0.86      0.88       254
    positive       0.86      0.89      0.88       246

    accuracy                           0.88       500
   macro avg       0.88      0.88      0.88       500
weighted avg       0.88      0.88      0.88       500


Custom Predictions:
I loved this movie! -> positive
It was the worst film ever. -> negative
The acting was okay, but story was boring. -> negative
